<a href="https://colab.research.google.com/github/HassanSamkari/Applied-Data-Science-Capstone/blob/master/Fashion_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [0]:
sales = pd.read_csv('/content/drive/My Drive/project for colab/data/sales_stock.csv')
products = pd.read_csv('/content/drive/My Drive/project for colab/data/products.csv')
positions = pd.read_csv('/content/drive/My Drive/project for colab/data/positions.csv')


In [5]:
sales.head()

,date_number,product_id,color_id,size_id,sales,stock
0,0,310130,78467031,7026791,1,93
1,0,310130,78467031,8015276,2,62
2,0,310130,78467031,452052101,1,18
3,0,310130,78467031,541898038,2,42
4,0,310130,78467031,558685710,3,113


In [6]:
products.head()

,product_id,family_id,subfamily_id,price
0,151926,396066037,335531561,25.95
1,213413,552529755,11509337,19.95
2,310130,679611953,533441312,12.95
3,373883,775013441,62518413,17.95
4,455200,998145072,490222156,29.95


In [8]:
positions.head()

,date_number,product_id,category_id,position
0,0,4450020,4461548,17
1,0,42147334,4461548,4
2,0,81131830,4461548,35
3,0,84035833,4461548,38
4,0,125252584,4461548,39


In [0]:
stay_columns=['product_id','price'] 
sales71=pd.DataFrame(sales71.groupby(stay_columns).sum()['sales']).reset_index()